# 2D Anisotropic Diffusion (non coercive case)

We consider the study and analysis of the eigenvalue problem associated to the following anisotropic diffusion operator
$$
    - \nabla \cdot \left( \mathbf{b} \otimes \mathbf{b} \nabla \phi \right) = \omega^2 \phi
$$
where $\mathbf{b} = (\iota, 1)$ is the normalized vector of the magnetic field.

In fact, the original problem is 3D and the iota profile is given as

$$
\iota(r) = 0.47262 + 0.32392 r + 0.49604 r^2 + 5.3991 . 10^{-6} r^3 - 3.165 . 10^{-5} r^4 + 4.7963 . 10^{-5} r^5 - 2.2824 . 10^{-5} r^6
$$

which is the LHD-$\iota$-profile. 

## Formal model

In [1]:
from sympy.core.containers import Tuple

from sympde.core     import Constant
from sympde.calculus import grad, dot
from sympde.topology import ScalarFunctionSpace
from sympde.topology import Domain
from sympde.topology import elements_of
from sympde.expr     import BilinearForm
from sympde.expr     import integral

DIM = 1
domain = Domain('\Omega', dim=DIM)

V = ScalarFunctionSpace('V', domain)

u,v = elements_of(V, names='u,v')

bx = Constant('bx')
by = Constant('by')
b  = Tuple(bx, by)

expr = integral(domain, dot(b, grad(v)) * dot(b, grad(u))) 
a = BilinearForm((u,v), expr)

## Symbolic expression

In [2]:
# imports from gelato
from gelato.expr     import gelatize
from gelato.printing import latex

We compute the GLT expression by calling the function **gelatize** for a given degree $p$

In [3]:
p = 3

glt = gelatize(a, p)

TypeError: Can't multiply sequence by non-integer of type 'dx'

In [4]:
# css style
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()